In [1]:
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.corpus import stopwords  
from nltk import word_tokenize 
from nltk.stem import WordNetLemmatizer  

import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer

# Get data

In [3]:
data=pd.read_csv(r'крипта_2022-01-01 00:00:00-2022-01-02 00:00:00.csv', index_col=False)
data = data.drop(columns=['Unnamed: 0'])
data = data.rename(columns={'text': 'posts'})
data

,date,id,owner_id,from_id,posts,comments,likes,reposts,views
0,2022-01-01 00:01:01,52584,-201880129,-201880129,"Дорогие инвесторы и трейдеры, вот и наступил 2...",16,195,13,12860
1,2022-01-01 08:23:59,8717,337679425,337679425,💰👍🏻 Не знаете чем себя занять? \nПопробуйте ра...,0,7,0,27
2,2022-01-01 07:30:00,1319396,-29231851,-29231851,ОЗЗИ ОСБОРН ПРОДАЕТ ЛЕТУЧИХ МЫШЕЙ В ФОРМЕ NFT\...,2,23,12,10046
3,2022-01-01 11:41:26,4029,77162208,77162208,Бац и мы в 2️⃣0️⃣2️⃣2️⃣❗️\n\n🔥Мои краткие итог...,4,26,0,668
4,2022-01-01 11:01:00,673154,-46730819,-46730819,ОЗЗИ ОСБОРН ПРОДАЕТ ЛЕТУЧИХ МЫШЕЙ В ФОРМЕ NFT\...,1,18,19,6139
5,2022-01-01 10:15:00,740351,-27680436,-27680436,А вы имеете крипту?,1,35,7,9807
6,2022-01-01 10:09:09,481,-164878976,-164878976,СТЕЙКИНГ КРИПТОВАЛЮТ\n\nЧто такое стейкинг кри...,0,3,5,97
7,2022-01-01 10:08:19,6232,-105162659,-105162659,Денис Борисов - Где мы. Полный прогноз на 2022...,0,20,8,951
8,2022-01-01 13:54:57,1854,35127141,35127141,31 декабря все подводили итоги уходящего 2021 ...,1,15,0,287
9,2022-01-01 13:47:57,14990,-192611461,-192611461,Как сохранить документ в данном формате (sgn)....,22,18,13,3577


# Filter for zero-impact news (likes > 0 and not epmty)

In [4]:
not_empty_flags = [len(post) > 0 for post in data.posts]
len(not_empty_flags)
data = data[not_empty_flags & data.likes > 0]
data

,date,id,owner_id,from_id,posts,comments,likes,reposts,views
0,2022-01-01 00:01:01,52584,-201880129,-201880129,"Дорогие инвесторы и трейдеры, вот и наступил 2...",16,195,13,12860
1,2022-01-01 08:23:59,8717,337679425,337679425,💰👍🏻 Не знаете чем себя занять? \nПопробуйте ра...,0,7,0,27
2,2022-01-01 07:30:00,1319396,-29231851,-29231851,ОЗЗИ ОСБОРН ПРОДАЕТ ЛЕТУЧИХ МЫШЕЙ В ФОРМЕ NFT\...,2,23,12,10046
5,2022-01-01 10:15:00,740351,-27680436,-27680436,А вы имеете крипту?,1,35,7,9807
6,2022-01-01 10:09:09,481,-164878976,-164878976,СТЕЙКИНГ КРИПТОВАЛЮТ\n\nЧто такое стейкинг кри...,0,3,5,97
8,2022-01-01 13:54:57,1854,35127141,35127141,31 декабря все подводили итоги уходящего 2021 ...,1,15,0,287
10,2022-01-01 13:30:00,72802,-140880848,-140880848,Почему децентрализация — это единственное важн...,4,87,34,7844
11,2022-01-01 12:51:02,125953,-110242131,-110242131,ПРОЕКТ ПО ИСЧЕЗНОВЕНИЮ ЖЕНЩИН \n \nЗдравствуйт...,8,29,49,5829
15,2022-01-01 19:58:00,13905,10481711,10481711,Ян Бедерман (Россия) и Беркант Чакыджи (Турция...,0,1,0,127
16,2022-01-01 19:30:00,10223307,-25232578,-25232578,Банк России продолжает давление на индустрию к...,14,23,24,17436


# Delete all links

In [5]:
data['posts'] = data['posts'].apply(lambda x: re.sub(r'http\S+', '', x, flags=re.MULTILINE))
data['posts'] = data['posts'].apply(lambda x: re.sub(r'\[id.+?\]', '', x, flags=re.MULTILINE))
data['posts'] = data['posts'].apply(lambda x: re.sub(r'\[club.+?\]', '', x, flags=re.MULTILINE))
data

,date,id,owner_id,from_id,posts,comments,likes,reposts,views
0,2022-01-01 00:01:01,52584,-201880129,-201880129,"Дорогие инвесторы и трейдеры, вот и наступил 2...",16,195,13,12860
1,2022-01-01 08:23:59,8717,337679425,337679425,💰👍🏻 Не знаете чем себя занять? \nПопробуйте ра...,0,7,0,27
2,2022-01-01 07:30:00,1319396,-29231851,-29231851,ОЗЗИ ОСБОРН ПРОДАЕТ ЛЕТУЧИХ МЫШЕЙ В ФОРМЕ NFT\...,2,23,12,10046
5,2022-01-01 10:15:00,740351,-27680436,-27680436,А вы имеете крипту?,1,35,7,9807
6,2022-01-01 10:09:09,481,-164878976,-164878976,СТЕЙКИНГ КРИПТОВАЛЮТ\n\nЧто такое стейкинг кри...,0,3,5,97
8,2022-01-01 13:54:57,1854,35127141,35127141,31 декабря все подводили итоги уходящего 2021 ...,1,15,0,287
10,2022-01-01 13:30:00,72802,-140880848,-140880848,Почему децентрализация — это единственное важн...,4,87,34,7844
11,2022-01-01 12:51:02,125953,-110242131,-110242131,ПРОЕКТ ПО ИСЧЕЗНОВЕНИЮ ЖЕНЩИН \n \nЗдравствуйт...,8,29,49,5829
15,2022-01-01 19:58:00,13905,10481711,10481711,Ян Бедерман (Россия) и Беркант Чакыджи (Турция...,0,1,0,127
16,2022-01-01 19:30:00,10223307,-25232578,-25232578,Банк России продолжает давление на индустрию к...,14,23,24,17436


# Delete texts without cyrilic

In [6]:
not_cyrilic_flags = [bool(re.search('[а-яА-Я]', post)) for post in data.posts]
len(not_cyrilic_flags)
# not_cyrilic_flags
data = data[not_cyrilic_flags]
data

,date,id,owner_id,from_id,posts,comments,likes,reposts,views
0,2022-01-01 00:01:01,52584,-201880129,-201880129,"Дорогие инвесторы и трейдеры, вот и наступил 2...",16,195,13,12860
1,2022-01-01 08:23:59,8717,337679425,337679425,💰👍🏻 Не знаете чем себя занять? \nПопробуйте ра...,0,7,0,27
2,2022-01-01 07:30:00,1319396,-29231851,-29231851,ОЗЗИ ОСБОРН ПРОДАЕТ ЛЕТУЧИХ МЫШЕЙ В ФОРМЕ NFT\...,2,23,12,10046
5,2022-01-01 10:15:00,740351,-27680436,-27680436,А вы имеете крипту?,1,35,7,9807
6,2022-01-01 10:09:09,481,-164878976,-164878976,СТЕЙКИНГ КРИПТОВАЛЮТ\n\nЧто такое стейкинг кри...,0,3,5,97
8,2022-01-01 13:54:57,1854,35127141,35127141,31 декабря все подводили итоги уходящего 2021 ...,1,15,0,287
10,2022-01-01 13:30:00,72802,-140880848,-140880848,Почему децентрализация — это единственное важн...,4,87,34,7844
11,2022-01-01 12:51:02,125953,-110242131,-110242131,ПРОЕКТ ПО ИСЧЕЗНОВЕНИЮ ЖЕНЩИН \n \nЗдравствуйт...,8,29,49,5829
15,2022-01-01 19:58:00,13905,10481711,10481711,Ян Бедерман (Россия) и Беркант Чакыджи (Турция...,0,1,0,127
16,2022-01-01 19:30:00,10223307,-25232578,-25232578,Банк России продолжает давление на индустрию к...,14,23,24,17436


# Drop duplicates posts

In [7]:
data = data.loc[data['posts'].drop_duplicates().index]
data.reset_index(drop=True, inplace=True)
data

,date,id,owner_id,from_id,posts,comments,likes,reposts,views
0,2022-01-01 00:01:01,52584,-201880129,-201880129,"Дорогие инвесторы и трейдеры, вот и наступил 2...",16,195,13,12860
1,2022-01-01 08:23:59,8717,337679425,337679425,💰👍🏻 Не знаете чем себя занять? \nПопробуйте ра...,0,7,0,27
2,2022-01-01 07:30:00,1319396,-29231851,-29231851,ОЗЗИ ОСБОРН ПРОДАЕТ ЛЕТУЧИХ МЫШЕЙ В ФОРМЕ NFT\...,2,23,12,10046
3,2022-01-01 10:15:00,740351,-27680436,-27680436,А вы имеете крипту?,1,35,7,9807
4,2022-01-01 10:09:09,481,-164878976,-164878976,СТЕЙКИНГ КРИПТОВАЛЮТ\n\nЧто такое стейкинг кри...,0,3,5,97
5,2022-01-01 13:54:57,1854,35127141,35127141,31 декабря все подводили итоги уходящего 2021 ...,1,15,0,287
6,2022-01-01 13:30:00,72802,-140880848,-140880848,Почему децентрализация — это единственное важн...,4,87,34,7844
7,2022-01-01 12:51:02,125953,-110242131,-110242131,ПРОЕКТ ПО ИСЧЕЗНОВЕНИЮ ЖЕНЩИН \n \nЗдравствуйт...,8,29,49,5829
8,2022-01-01 19:58:00,13905,10481711,10481711,Ян Бедерман (Россия) и Беркант Чакыджи (Турция...,0,1,0,127
9,2022-01-01 19:30:00,10223307,-25232578,-25232578,Банк России продолжает давление на индустрию к...,14,23,24,17436


# Filter posts with big numbers of emojis

In [8]:
emoji_count = [len(re.findall(u'[\U0001f34a-\U0001f650]', x)) for x in data.posts]
data = data[[flag < 10 for flag in emoji_count]]
data.reset_index(drop=True, inplace=True)
data

,date,id,owner_id,from_id,posts,comments,likes,reposts,views
0,2022-01-01 00:01:01,52584,-201880129,-201880129,"Дорогие инвесторы и трейдеры, вот и наступил 2...",16,195,13,12860
1,2022-01-01 08:23:59,8717,337679425,337679425,💰👍🏻 Не знаете чем себя занять? \nПопробуйте ра...,0,7,0,27
2,2022-01-01 07:30:00,1319396,-29231851,-29231851,ОЗЗИ ОСБОРН ПРОДАЕТ ЛЕТУЧИХ МЫШЕЙ В ФОРМЕ NFT\...,2,23,12,10046
3,2022-01-01 10:15:00,740351,-27680436,-27680436,А вы имеете крипту?,1,35,7,9807
4,2022-01-01 10:09:09,481,-164878976,-164878976,СТЕЙКИНГ КРИПТОВАЛЮТ\n\nЧто такое стейкинг кри...,0,3,5,97
5,2022-01-01 13:54:57,1854,35127141,35127141,31 декабря все подводили итоги уходящего 2021 ...,1,15,0,287
6,2022-01-01 13:30:00,72802,-140880848,-140880848,Почему децентрализация — это единственное важн...,4,87,34,7844
7,2022-01-01 12:51:02,125953,-110242131,-110242131,ПРОЕКТ ПО ИСЧЕЗНОВЕНИЮ ЖЕНЩИН \n \nЗдравствуйт...,8,29,49,5829
8,2022-01-01 19:58:00,13905,10481711,10481711,Ян Бедерман (Россия) и Беркант Чакыджи (Турция...,0,1,0,127
9,2022-01-01 19:30:00,10223307,-25232578,-25232578,Банк России продолжает давление на индустрию к...,14,23,24,17436


# Import StopWords and Packets

In [9]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /home/danil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/danil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/danil/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/danil/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [10]:
with open('all_stop_words.txt', 'r') as file:
    stop_words = file.read()
    stop_words = stop_words.split()

stop_words
len(stop_words)

1506

# Lemmatization

In [11]:
mystem = Mystem() 

In [12]:
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in stop_words\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

In [13]:
data['cleaned_posts'] = data.posts.apply(preprocess_text)
data

,date,id,owner_id,from_id,posts,comments,likes,reposts,views,cleaned_posts
0,2022-01-01 00:01:01,52584,-201880129,-201880129,"Дорогие инвесторы и трейдеры, вот и наступил 2...",16,195,13,12860,дорогой инвестор трейдер наступать 2022 желать...
1,2022-01-01 08:23:59,8717,337679425,337679425,💰👍🏻 Не знаете чем себя занять? \nПопробуйте ра...,0,7,0,27,💰 👍 🏻 знать занимать попробовать размещать рек...
2,2022-01-01 07:30:00,1319396,-29231851,-29231851,ОЗЗИ ОСБОРН ПРОДАЕТ ЛЕТУЧИХ МЫШЕЙ В ФОРМЕ NFT\...,2,23,12,10046,оззи осборн продавать летучий мышь форма nft о...
3,2022-01-01 10:15:00,740351,-27680436,-27680436,А вы имеете крипту?,1,35,7,9807,крипта
4,2022-01-01 10:09:09,481,-164878976,-164878976,СТЕЙКИНГ КРИПТОВАЛЮТ\n\nЧто такое стейкинг кри...,0,3,5,97,стейкинг криптовалюта стейкинг криптовалюта ст...
5,2022-01-01 13:54:57,1854,35127141,35127141,31 декабря все подводили итоги уходящего 2021 ...,1,15,0,287,31 декабрь подводить итог уходить 2021 январь ...
6,2022-01-01 13:30:00,72802,-140880848,-140880848,Почему децентрализация — это единственное важн...,4,87,34,7844,децентрализация — единственный свойство крип...
7,2022-01-01 12:51:02,125953,-110242131,-110242131,ПРОЕКТ ПО ИСЧЕЗНОВЕНИЮ ЖЕНЩИН \n \nЗдравствуйт...,8,29,49,5829,проект исчезновение женщина здравствовать анон...
8,2022-01-01 19:58:00,13905,10481711,10481711,Ян Бедерман (Россия) и Беркант Чакыджи (Турция...,0,1,0,127,бедерман россия беркант чакыджи турция кц артэ...
9,2022-01-01 19:30:00,10223307,-25232578,-25232578,Банк России продолжает давление на индустрию к...,14,23,24,17436,банк россия продолжать давление индустрия крип...


# Drop duplicates posts

In [14]:
data = data.loc[data['cleaned_posts'].drop_duplicates().index]
data.reset_index(drop=True, inplace=True)
data

,date,id,owner_id,from_id,posts,comments,likes,reposts,views,cleaned_posts
0,2022-01-01 00:01:01,52584,-201880129,-201880129,"Дорогие инвесторы и трейдеры, вот и наступил 2...",16,195,13,12860,дорогой инвестор трейдер наступать 2022 желать...
1,2022-01-01 08:23:59,8717,337679425,337679425,💰👍🏻 Не знаете чем себя занять? \nПопробуйте ра...,0,7,0,27,💰 👍 🏻 знать занимать попробовать размещать рек...
2,2022-01-01 07:30:00,1319396,-29231851,-29231851,ОЗЗИ ОСБОРН ПРОДАЕТ ЛЕТУЧИХ МЫШЕЙ В ФОРМЕ NFT\...,2,23,12,10046,оззи осборн продавать летучий мышь форма nft о...
3,2022-01-01 10:15:00,740351,-27680436,-27680436,А вы имеете крипту?,1,35,7,9807,крипта
4,2022-01-01 10:09:09,481,-164878976,-164878976,СТЕЙКИНГ КРИПТОВАЛЮТ\n\nЧто такое стейкинг кри...,0,3,5,97,стейкинг криптовалюта стейкинг криптовалюта ст...
5,2022-01-01 13:54:57,1854,35127141,35127141,31 декабря все подводили итоги уходящего 2021 ...,1,15,0,287,31 декабрь подводить итог уходить 2021 январь ...
6,2022-01-01 13:30:00,72802,-140880848,-140880848,Почему децентрализация — это единственное важн...,4,87,34,7844,децентрализация — единственный свойство крип...
7,2022-01-01 12:51:02,125953,-110242131,-110242131,ПРОЕКТ ПО ИСЧЕЗНОВЕНИЮ ЖЕНЩИН \n \nЗдравствуйт...,8,29,49,5829,проект исчезновение женщина здравствовать анон...
8,2022-01-01 19:58:00,13905,10481711,10481711,Ян Бедерман (Россия) и Беркант Чакыджи (Турция...,0,1,0,127,бедерман россия беркант чакыджи турция кц артэ...
9,2022-01-01 19:30:00,10223307,-25232578,-25232578,Банк России продолжает давление на индустрию к...,14,23,24,17436,банк россия продолжать давление индустрия крип...


In [22]:
data.to_csv('cleaned_data.csv')
data.head().loc[0].cleaned_posts

'дорогой инвестор трейдер наступать 2022 желать успешный сделка зеленый портфель вылечиваться fomo думать голова эмоция покупать фундаментальный компания закредитовать помойка ловить икс крипта делать адепт shiba inu крепкий здоровье нерв остальное свой сторона радовать годный мемаса видос скрин профи пульс портфель зеленый число новый'